# Разработка решения для увеличения покупательской активности постоянных клиентов

По условиям поставленного задания, нам необходимо произвести маркировку уровня финансовой активности постоянных клиентов интернет-магазина «В один клик», собрать признаки клиентов в группы, создать модель, которая подскажет вероятность снижения покупательской активности клиента в следующие три месяца, выделить сегменты покупателей и разработать для них персонализированные предложения. Для анализа нам предоставлено четыре файла с данными.

В файле `market_file.csv` содержатся следующие данные: `id` — номер покупателя в БД, `Покупательская активность` — целевой признак, `Тип сервиса` — уровень сервиса, `Разрешить сообщать` — согласие на рассылку дополнительной информации, `Маркет_актив_6_мес` — количество маркетинговых коммуникаций за полгода, `Маркет_актив_тек_мес` — количество маркетинговых коммуникаций в текущем месяце, `Длительность` — количество дней с момента регистрации на сайте, `Акционные_покупки` — среднемесячная доля покупок по акции от общего числа покупок за полгода, `Популярная_категория` — самая популярная категория покупок за полгода, `Средний_просмотр_категорий_за_визит` — сколько категорий в среднем просмотрел покупатель за визит в течение месяца, `Неоплаченные_продукты_штук_квартал` — количество неоплаченных товаров в корзине за квартал, `Ошибка_сервиса` — количество сбоев, с которыми столкнулся покупатель во время посещения сайта и `Страниц_за_визит` — среднее количество просмотренных страниц за визит в течение квартала.

В файле `market_money.csv` содержатся следующие данные: `id` — номер покупателя в БД, `Период` — название периода, в который зафиксирована выручка, `Выручка` — сумма выручки за период.

В файле `market_time.csv` содержатся следующие данные: `id` — номер покупателя в БД, `Период` — название периода, в который зафиксировано время посещения сайта, `минут` — значение времени проведенного клиентом на сайте в минутах.

В файле `money.csv` содержатся следующие данные: `id` — номер покупателя в БД, `Прибыль` — среднемесячная прибыль магазина от покупателя за квартал.

В рамках проекта мы планируем произвести загрузку данных, произвести первичный анализ, предобработку и детальное рассмотрение каждого параметра имеющихся данных. Мы произведем устранение пропусков, если в этом есть необходимость, избавимся от дубликатов, приведем данные к соответствующим типам, удалим аномалии и выбросы или опишем их, произведем переименование столбцов и объединение таблиц `market_file.csv`, `market_money.csv` и `market_time.csv`, корреляционный анализ признаков.

Далее нами будет создан пайплайн для подбора лучшей модели. На основе результатов работы модели и данных из таблицы `money.csv` мы выполним сегментацию покупателей и предложим рекомендации для увеличения покупательской активности.

## Загрузка данных

Для начала произведем загрузку необходимых библиотек (`NumPy`, `pandas`, `phik`, `seaborn` и `shap`) и модулей (`path` из системной библиотеки `os`, подмодуля `SMOTENC` из модуля `over_sampling` библиотеки `imbalanced-learn`, подмодулей `HTML` и `display` из модуля `display` библиотеки `IPython`, `pyplot` из библиотеки `Matplotlib`, модуля `express` из библиотеки `plotly`, множества подмодулей из модулей библиотеки `scikit-learn`), произведем настройку отображения графиков по центру страницы блокнота с помощью CSS и формата чисел с плавающей точкой в `pandas`.

In [ ]:
# Устанавливаем дополнительные библиотеки, которых нет
# на платформе.
# Импортируем библиотеки и модули.
from os import path as ospath

import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import HTML, display
from matplotlib import pyplot as plt
from plotly import express as px
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

try:
    import phik
except ModuleNotFoundError:
    !pip install phik -q
    import phik

try:
    import shap
except ModuleNotFoundError:
    !pip install shap -q
    import shap

try:
    from imblearn.over_sampling import SMOTENC
except ModuleNotFoundError:
    !pip install imbalanced-learn -q
    !pip install imblearn -q
    from imblearn.over_sampling import SMOTENC

In [ ]:
# Опция для отображения чисел с плавающей точкой в удобном формате.
pd.set_option("float_format", "{:,.3f}".format)
pd.set_option("display.max_columns", None)

# Опция для отображения графиков по центру страницы блокнота.
display(
    HTML(
        "<style>.output_png, .jp-RenderedImage {display: table-cell;"
        "text-align: center; vertical-align: middle;}</style>"
    )
)

Для загрузки данных из возможных источников воспользуемся собственной функцией.

In [ ]:
# Функция для загрузки данных из CSV-файла локально или удаленно.


def load_data(path, col_sep=",", float_sep=".", data=None):
    """Функция для загрузки данных из CSV-файла локально или удаленно."""
    for instance in path, f"/{path}":
        if ospath.exists(instance):
            data = pd.read_csv(
                instance, sep=col_sep, decimal=float_sep, engine="python"
            )
            break

    if data is None:
        try:
            data = pd.read_csv(
                f"https://code.s3.yandex.net/{path}",
                sep=col_sep,
                decimal=float_sep,
                engine="python",
            )
        except IOError:
            print("Файл с данными не найден.")

    return data

Загрузим таблицу `market_file.csv`.

In [ ]:
# Если возможно, загружаем данные, выводим информацию о таблице,
# ее первые 20 строк или ошибку, сохраянем данные в датафрейм df_mfile.
df = load_data("datasets/market_file.csv")

if df is not None:
    df.info()
    display(df.head(20))
    df_mfile = df

На первый взгляд, типы данных в столбцах соответствуют сохраненной в них информации, хотя очевидно, что в данных есть опечатки (например, *стандартт*), пропусков в данных нет. Стоит отметить, что в столбцах `Покупательская активность`, `Тип сервиса` и `Разрешить сообщать` содержатся бинарные признаки.

Загрузим таблицу `market_money.csv`.

In [ ]:
# Если возможно, загружаем данные, выводим информацию о таблице,
# ее первые 20 строк или ошибку, сохраянем данные в датафрейм
# df_mmoney.
df = load_data("datasets/market_money.csv")

if df is not None:
    df.info()
    display(df.head(20))
    df_mmoney = df

На первый взгляд, типы данных в столбцах соответствуют сохраненной в них информации, пропусков в данных нет.

Загрузим таблицу `market_time.csv`.

In [ ]:
# Если возможно, загружаем данные, выводим информацию о таблице,
# ее первые 20 строк или ошибку, сохраянем данные в датафрейм df_mtime.
df = load_data("datasets/market_time.csv")

if df is not None:
    df.info()
    display(df.head(20))
    df_mtime = df

На первый взгляд, типы данных в столбцах соответствуют сохраненной в них информации, хотя очевидно, что в данных есть опечатки (например, *предыдцщий_месяц*), пропусков в данных нет.

Загрузим таблицу `money.csv`.

In [ ]:
# Если возможно, загружаем данные, выводим информацию о таблице,
# ее первые 20 строк или ошибку, сохраянем данные в датафрейм df_money.
df = load_data("datasets/money.csv", ";", ",")

if df is not None:
    df.info()
    display(df.head(20))
    df_money = df

На первый взгляд, типы данных в столбцах соответствуют сохраненной в них информации, пропусков в данных нет.

**Вывод**: мы произвели загрузку необходимых модулей и билиотек для дальнейшей работы, воспользовались собственной функцией для загрузки файлов с данными и произвели первичный (визуальный) анализ первых 20 строк в четырех таблицах. Нами были выявлены опечатки в некоторых столбцах. Исходя из количества записей в таблицах, мы можем предположить, что сервис предоставил информацию о 1 300 клиентах и их покупательской активности на сайте за три месяца. При этом в трех столбцах первой таблицы содержатся данные за полгода. Во всех четырех таблицах содержится столбец `id`, который позволяет установить однозначное соотвествие между клиентом и информацией, собранной о нем в различных таблицах.

## Предобработка данных

Для начала произведем переименование столбцов во всех таблицах, заменив их на английские, чтобы потом было проще писать код в одной раскладке.

In [ ]:
# Переименовываем столбцы в датафрейме df_mfile.
df_mfile = df_mfile.rename(
    columns={
        "Покупательская активность": "same_level_activity",
        "Тип сервиса": "is_premium",
        "Разрешить сообщать": "allow_marketing",
        "Маркет_актив_6_мес": "marketing_6_months",
        "Маркет_актив_тек_мес": "marketing_this_month",
        "Длительность": "days_since_registration",
        "Акционные_покупки": "deal_share",
        "Популярная_категория": "popular_category",
        "Средний_просмотр_категорий_за_визит": "average_category_per_visit",
        "Неоплаченные_продукты_штук_квартал": "unpaid_goods_quarter",
        "Ошибка_сервиса": "errors",
        "Страниц_за_визит": "pages_per_visit",
    }
)

# Переименовываем столбцы в датафрейме df_mmoney.
df_mmoney = df_mmoney.rename(
    columns={"Период": "time_period", "Выручка": "earnings_per_period"}
)

# Переименовываем столбцы в датафрейме df_mtime.
df_mtime = df_mtime.rename(columns={"Период": "is_current_period", "минут": "minutes"})

# Переименовываем столбцы в датафрейме df_money.
df_money = df_money.rename(columns={"Прибыль": "earnings_per_quarter"})

Поскольку мы определили, что в столбцах `same_level_activity`, `is_premium` и `allow_marketing` датафрейма `df_mfile` содержатся бинарные данные, убедимся, что в них, действительно, содержатся только два значения. Также проверим, что только два значения содержатся в столбце `is_current_period` датафрейма `df_mtime`.

In [ ]:
# Проверяем уникальные значения в бинарных столбцах.
print(df_mfile["same_level_activity"].unique())
print(df_mfile["is_premium"].unique())
print(df_mfile["allow_marketing"].unique())
print(df_mtime["is_current_period"].unique())

Мы видим, что в двух столбцах их трех датафрейма `df_mfile` содержатся только два значения, а в третьем за счет опечатки воникает третье значение. Сменим тип данных на булев и заодно сменим регистр в столбце `popular_category`. В датафрейме `df_mtime` в столбце `is_current_period` также содержится всего два значения (одно с опечаткой), потому мы также можем сменить тип данных в нем на булев.

In [ ]:
# Меняем регистр на строчный и приводим три столбца
# к булевому типу данных в датафрейме df_mfile.
df_mfile[df_mfile.select_dtypes("object").columns] = (
    df_mfile[df_mfile.select_dtypes("object").columns]
    .apply(lambda x: x.str.lower())
    .replace(
        {
            "прежний уровень": True,
            "премиум": True,
            "да": True,
            "снизилась": False,
            "стандартт": False,
            "стандарт": False,
            "нет": False,
        }
    )
)

# Приводим столбец к булевому типу данных в датафрейме df_mtime.
df_mtime[df_mtime.select_dtypes("object").columns] = df_mtime[
    df_mtime.select_dtypes("object").columns
].replace({"текущий_месяц": True, "предыдцщий_месяц": False})

Убедимся, что столбцы были верно переименованы и типы данных изменились на булев после проведения операций.

In [ ]:
# Выводим информацию по четырем датафреймам.
df_mfile.info()
print()
df_mmoney.info()
print()
df_mtime.info()
print()
df_money.info()

Убедимся, что в наших данных не содержится полных дубликатов.

In [ ]:
# Количество полных дубликатов в четырех датафреймах.
print(
    f"Количество полных дубликатов в датафрейме df_mfile: "
    f"{df_mfile.duplicated().sum()}."
)
print(
    f"Количество полных дубликатов в датафрейме df_mmoney: "
    f"{df_mmoney.duplicated().sum()}."
)
print(
    f"Количество полных дубликатов в датафрейме df_mtime: "
    f"{df_mtime.duplicated().sum()}."
)
print(
    f"Количество полных дубликатов в датафрейме df_money: "
    f"{df_money.duplicated().sum()}."
)

Проверим также, что во всех датафреймах содержится ровно 1 300 уникальных номеров покупателей в БД.

*Замечание. На данном этапе, мы не станем проверять, совпадают ли номера по всем четырем датафреймам и предположим, что данные собраны и выгружены корректно.*

In [ ]:
# Количество уникальных номеров покупателей в датафреймах.
print(
    f"Количество уникальных значений в столбце id в датафрейме df_mfile: "
    f"{df_mfile['id'].nunique():,.0f}.".replace(",", " ")
)
print(
    f"Количество уникальных значений в столбце id в датафрейме df_mmoney: "
    f"{df_mmoney['id'].nunique():,.0f}.".replace(",", " ")
)
print(
    f"Количество уникальных значений в столбце id в датафрейме df_mtime: "
    f"{df_mtime['id'].nunique():,.0f}.".replace(",", " ")
)
print(
    f"Количество уникальных значений в столбце id в датафрейме df_money: "
    f"{df_money['id'].nunique():,.0f}.".replace(",", " ")
)

**Вывод**: мы произвели переименование столбцов в четырех датафреймах, сменили регистр в одном столбце датафрейма `df_mfile` и сменили тип данных в четырех столбцах двух датафреймов на булев. Мы выявили, что в данных не содержится полных дубликатов, что в каждом датафрейме есть данные ровно по 1 300 уникальным номерам покупателей в БД. Мы сделали допущение, что эти 1 300 номеров совпадают между датафреймами, поскольку данные в целом отличаются хорошим уровнем подготовки.

## Исследовательский анализ данных

Проведем исследовательский анализ данных для всех столбцов в четырех датафреймах, кроме столбца `id`.

Для начала создадим функцию для построения круговых диаграмм для столбцов с булевым типом данных.

In [ ]:
# Функция для создания круговых диаграммы для булевых типов данных.


def tasty_pies(df_col_name, labels, title):
    """Функция для отрисовки круговых диаграмм."""
    fig = px.pie(
        df_col_name.value_counts(), names=labels, values=df_col_name.value_counts()
    )
    fig.update_layout(title_text=title, title_x=0.5, showlegend=False)
    fig.show()

Посмотрим на столбец `same_level_activity` датафрейма `df_mfile`.

In [ ]:
# Круговая диаграмма через функцию.
tasty_pies(
    df_mfile["same_level_activity"],
    ["Прежний уровень", "Снизился"],
    "Уровень покупательской активности",
)

У большинства клиентов (802, 61,7 %) уровень покупательской активности остался на прежнем уровне, однако у 498 клиентов (38,3 %) он снизился.

Посмотрим на столбец `is_premium` датафрейма `df_mfile`.

In [ ]:
# Круговая диаграмма через функцию.
tasty_pies(
    df_mfile["is_premium"],
    ["Нет подписки", "Есть подписка"],
    "Наличие премиум подписки",
)

У большинства клиентов нет премиальной подписки (924, 71,1 %). Подписка имеется только у 376 клиентов (28,9 %) в выборке.

Посмотрим на столбец `allow_marketing` датафрейма `df_mfile`.

In [ ]:
# Круговая диаграмма через функцию.
tasty_pies(
    df_mfile["allow_marketing"],
    ["Разрешить", "Запретить"],
    "Дополнительные маркетинговые коммуникации",
)

Большая часть клиентов (962, 74 %) предпочитает получать дополнительную маркетинговую коммуникацию по интересующим их товарам. 338 клиентов в выборке (26 %) отказались от данной опции.

Посмотрим на столбец `is_current_period` датафрейма `df_mtime`.

In [ ]:
# Круговая диаграмма через функцию.
tasty_pies(
    df_mtime["is_current_period"],
    ["Текущий месяц", "Предыдущий месяц"],
    "Период посещения сайта",
)

Ожидаемо, данные в этом столбце распределены пополам по двум периодам (предыдущий и текущий).

Также в предоставленных нам данных имеется два столбца со строковыми значениями (`popular_category` и `time_period`), которые мы также можем рассмотреть на круговых диаграммах.

In [ ]:
# Круговая диаграмма через функцию.
tasty_pies(
    df_mfile["popular_category"],
    [
        "Товары для детей",
        "Домашний текстиль",
        "Косметика и аксессуары",
        "Техника для красоты и здоровья",
        "Мелкая бытовая техника и электроника",
        "Кухонная посуда",
    ],
    "Популярные категории",
)

Наиболее популярными категориями у посетителей сайта являются **Товары для детей** (330, 25,4 %), **Домашний текстиль** (251, 19,3 %) и **Косметика и аксессуары** (223, 17,2 %). Далее идут **Техника для красоты и здоровья** (184, 14,2 %), **Мелкая бытовая техника и электроника** (174, 13,4 %) и **Кухонная посуда** (138, 10,6 %). В целом список категорий указывает на то, что целевой аудиторией сайта являются семейные женщины с детьми. *Больше 100% в связи с округлением долей.*

Убедимся, что в столбце `time_period` датафрейма `df_mmoney` данные распределены равномерно между тремя периодами.

In [ ]:
# Круговая диаграмма через функцию.
tasty_pies(
    df_mmoney["time_period"],
    ["Предыдущий месяц", "Месяц перед предыдущим", "Текущий месяц"],
    "Период",
)

Ожидаемо, данные в этом столбце распределены равномерно по трем периодам. *Менее 100% в связи с округлением долей.*

Теперь рассмотрим столбцы с количественными данными через функцию.

In [ ]:
def boring_bars(df_name, some_column, title):
    """Функция для построения гистограмм и «ящиков с усами»"""
    # Строим гистограмму для столбца.
    df_name.hist(column=some_column, bins=20, figsize=(17, 5))
    plt.title(title)
    plt.show()

    # Строим «ящик с усами» для столбца.
    df_name.boxplot(some_column, figsize=(18, 3), vert=False).set_title(title)
    plt.tick_params(left=False)
    plt.yticks([1], "")
    plt.show()

    # Выводим описание данных.
    display(df_name[some_column].describe())

Для начала посмотрим на столбец `marketing_6_months`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(
    df_mfile, "marketing_6_months", "Количество маркетинговых коммуникаций за полгода"
)

В среднем клиенты интернет-магазина получали 4,254 маркетинговых коммуникаций в месяц (медианное значение — 4,2). Максимальное количество маркетинговых коммуникаций — 6,6, минимальное — 0,9.

Теперь посмотрим на столбец `marketing_this_month`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(
    df_mfile,
    "marketing_this_month",
    "Количество маркетинговых коммуникаций в текущем месяце",
)

В данном столбце данные принимают одно из трех значений (3, 4 и 5). Соотвественно, медианное значение — 4.

Теперь посмотрим на столбец `days_since_registration`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(
    df_mfile, "days_since_registration", "Количество дней с момента регистрации"
)

Все пользователи в выборке зарегистрировались на сайте минимум 110 дней назад, а некоторые почти 3 года назад (более 1 000 дней на сайте). Средний «стаж» на сайте составляет почти 2 года (601,898 дней), медианный чуть выше — 606 дней.

Теперь посмотрим на столбец `deal_share`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(df_mfile, "deal_share", "Доля купленных акционных товаров")

В среднем пользователи сайта покупают 32 % товаров по акции (медианное значение ниже, 24 %). Есть пользователи, которые, видимо, покупают товары независимо оттого, участвуют ли те в акциях или нет (минимальное значение акционных товаров равно 0). Однако находятся и те, кто приобретают 99 % товаров по акциям, но пользователей, которые приобретают 60 % товаров и более по акции все же единицы (выбросы справа в «ящике с усами»).

Теперь посмотрим на столбец `average_category_per_visit`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(
    df_mfile,
    "average_category_per_visit",
    "Среднее количество просмотренных категорий за визит в текущем месяце",
)

Поскольку в нашей выборке представлено всего 6 категорий (вероятно, это все категории сайта), то значение строго ограничено 1 слева и 6 справа. В среднем посетители сайта смотрели 3,27 категорий за визит (медианное значение — 3).

Теперь посмотрим на столбец `unpaid_goods_quarter`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(
    df_mfile,
    "unpaid_goods_quarter",
    "Количество неоплаченных товаров в корзине за квартал",
)

В среднем у покупателей за три месяца накопилось 2,84 неоплаченных товара (медианное значение — 3), хотя есть и те, у кого нет ни одного неоплаченного товара, а также те у кого есть 9 и 10 неоплаченных товаров. Возможно, люди просто сохраняют товары в корзине и следят за динамикой цены, чтобы приобрести их, когда предложение будет более выгодным, чем обычно.

Теперь посмотрим на столбец `errors`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(
    df_mfile,
    "errors",
    "Количество сбоев, с которыми столкнулся посетитель во время визита сайта",
)

Предположительно, количество ошибок в данном столбце отражает количество ошибок за все время пользования сайтом, потому что в среднем пользователи сталкивались с ошибкой 4,185 раз (медианное значение — 4). Для каждого визита количество ошибок было бы слишком велико. Отдельные пользователи не столкнулись ни с одной ошибкой, а некоторые столкнулись даже с 9.

Теперь посмотрим на столбец `pages_per_visit`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(
    df_mfile, "pages_per_visit", "Среднее количество страниц за визит за квартал"
)

В среднем пользователи при каждом визите посещали 8,177 страницу (медианное значение — 8). Некоторые пользователи просмотрели только 1 страницу, а отдельные пользователи посещали до 20 страниц за один визит.

Теперь посмотрим на столбец `earnings_per_quarter` датафрейма `df_money`.

In [ ]:
# Гистограмма, «ящик с усами» и описание данных через функцию.
boring_bars(df_money, "earnings_per_quarter", "Сумма выручки за квартал с покупателя")

Нам неизвестна размерность выручки, но мы можем предположить, что она дана в тысячах рублей. В среднем покупатели приносили 3 997 ₽ магазину (медианное значение — 4 045 ₽). Отдельные покупатели принесли около 1 000 ₽ (минимально — 860 ₽), но были и супер-покупатели, которые принесли выручку более чем 7 000 ₽ за квартал.

Теперь посмотрим на столбец `earnings_per_period` датафрейма `df_mmoney`.

In [ ]:
# Строим диаграмму рассеивания для столбца.
fig = px.scatter(
    df_mmoney,
    y="earnings_per_period",
    color="time_period",
    labels={
        "earnings_per_period": "Выручка за период",
        "time_period": "Период, в который зафиксирована выручка",
    },
    title="Выручка от пользователей в различные периоды",
)
fig.show()

# Выводим описание данных по периодам.
display(
    df_mmoney["earnings_per_period"][
        df_mmoney["time_period"] == "текущий_месяц"
    ].describe()
)
display(
    df_mmoney["earnings_per_period"][
        df_mmoney["time_period"] == "предыдущий_месяц"
    ].describe()
)
(
    df_mmoney["earnings_per_period"][
        df_mmoney["time_period"] == "препредыдущий_месяц"
    ].describe()
)

Мы видим одно аномально большое значение выручки в текущем месяце (более 106 тыс. руб.). В шести наблюдениях мы видим нулевую выручку в предыдущем или в месяце перед предыдущим.

Максимальное значение выручки без аномального значения составляет 7 799,4 ₽, минимальное (без нулевых) — 2 758,7 ₽. Медианное значение выручки — \~ 5 тыс. руб. На среднее значение мы не смотрим, поскольку оно завышено за счет аномальных данных или занижено за счет нулевых показателей выручки.

По условиям задачи руководства мы должны отобрать клиентов, которые совершали покупки в течение квартала, то есть мы можем удалить из рассмотрения наблюдения с нулевой выручкой, а также клиента с аномально высокой выручкой. Нам известны индексы наблюдений (0, 2, 28, 29, 34, 35 и 98). Выясним номера клиентов в клиентской БД и удалим данные из всех четырех датафреймов.

In [ ]:
# Выводим id для удаления в будущем.
print(
    f"id клиентов с нулевыми покупками: "
    f"{list(df_mmoney.iloc[i][0] for i in [0, 2, 28, 29, 34, 35])}."
)
print(f"id клиента с аномально большими покупками: " f"{df_mmoney.iloc[98][0]}.")

Оказалось, что только три клиента из нашей выборки не совершали покупок в предыдущем и в месяце перед предыдущим. Таким образом, мы можем удалить из рассмотрения всего четыре записи из 1 300 / восемь из 2 600 / двенадцать из 3 900 (\~ 0,003 % данных).

Перед удаление данных посмотрим на столбец `minutes` датафрейма `df_mtime`.

In [ ]:
# Строим диаграмму рассеивания для столбца.
fig = px.histogram(
    df_mtime,
    x="minutes",
    color="is_current_period",
    labels={"minutes": "Минуты на сайте", "is_current_period": "Текущий период"},
    title="Минуты на сайте в текущий и предыдущий периоды",
    barmode="overlay",
)
fig.show()

# Выводим описание данных по периодам.
display(df_mtime["minutes"][df_mtime["is_current_period"] == True].describe())
(df_mtime["minutes"][df_mtime["is_current_period"] == False].describe())

Аномальных значений в данном столбце не обнаружено. Минимальное время нахождения на сайте снизилось на одну минуту за месяц, среднее время снизилось с 13,468 минут (\~ 13 минут 28 секунд) до 13,205 минут (\~ 13 минут 12 секунд). Медианное и максимальное время на сайте остались без изменений (13 минут и 23 минуты соотвественно).

Теперь мы можем удалить выявленные ранее аномальные данные.

In [ ]:
# Удаляем плохие записи из датафрейма df_mfile.
df_mfile = df_mfile[
    (
        (df_mfile.id != 215348)
        & (df_mfile.id != 215357)
        & (df_mfile.id != 215359)
        & (df_mfile.id != 215380)
    )
]

# Удаляем плохие записи из датафрейма df_mmoney.
df_mmoney = df_mmoney[
    (
        (df_mmoney.id != 215348)
        & (df_mmoney.id != 215357)
        & (df_mmoney.id != 215359)
        & (df_mmoney.id != 215380)
    )
]

# Удаляем плохие записи из датафрейма df_mtime.
df_mtime = df_mtime[
    (
        (df_mtime.id != 215348)
        & (df_mtime.id != 215357)
        & (df_mtime.id != 215359)
        & (df_mtime.id != 215380)
    )
]

# Удаляем плохие записи из датафрейма df_money.
df_money = df_money[
    (
        (df_money.id != 215348)
        & (df_money.id != 215357)
        & (df_money.id != 215359)
        & (df_money.id != 215380)
    )
]

**Вывод**: мы рассмотрели данные во всех столбцах всех четырех датафреймов, произвели статистический анализ распределения данных, удалили одну аномалию и три записи о клиентах, которые не совершали покупки на протяжении двух месяцев. Среднестатистический клиент интернет-магазина «В один клик» сохранил прежний уровень покупательской активности, не имеет премиальной подписки на сервис, дал согласие на получение дополнительной маркетинговой коммуникации по интересующим его товарам, приобрел товары в категории «Товары для детей», «Домашний текстиль» или «Косметика и аксессуары», в среднем имел чуть более 4 коммуникаций от маркетинговой службы интернет магазина (4 коммуникации в текущем месяце), зарегистрировался на сайте около 600 дней назад, \~ 24 % товаров приобрел по акции, просматривал 3 категории за визит на сайт в текущем месяце, имеет \~ 3 неоплаченных товара в корзине за квартал, столкнулся с \~ 4 сбоями во время визита сайта, просмотрел \~ 8 страниц товаров за визит за квартал, принес магазину выручку \~ 4 тыс. руб., провел на сайте \~ 13 минут за визит.

## Объединение таблиц

Произведем объединение трех датафреймов `df_mfile`, `df_mmoney` и `df_mtime` в единый датафрейм. При объединении нам необходимо учесть, что количество строк в первом датафрейме в три раза меньше чем во втором и в два раза меньше, чем в третьем в связи с тем, что данные по различным временным периодам собраны в одном столбце, а не в нескольких. Наиболее простым способом уровнять количество строк в датафреймах нам видится разбиение датафреймов по периодам и их последующее объединение по `id`.

Для начала выполним манипуляции с датафреймом `df_mmoney`.

In [ ]:
# Разбиваем df_mmoney на три датафрейма.
df_mmoney_current = df_mmoney[df_mmoney.time_period == "текущий_месяц"]

df_mmoney_last = df_mmoney[df_mmoney.time_period == "предыдущий_месяц"]

df_mmoney_before_last = df_mmoney[df_mmoney.time_period == "препредыдущий_месяц"]

# Удаляем ненужный столбец и переименовываем нужный.
df_mmoney_current = df_mmoney_current.drop("time_period", axis=1).rename(
    columns={"earnings_per_period": "earnings_this_month"}
)

df_mmoney_last = df_mmoney_last.drop("time_period", axis=1).rename(
    columns={"earnings_per_period": "earnings_last_month"}
)

df_mmoney_before_last = df_mmoney_before_last.drop("time_period", axis=1).rename(
    columns={"earnings_per_period": "earnings_before_last"}
)

# Объединяем три датафрейма обратно.
df_mmoney = df_mmoney_current.merge(
    df_mmoney_last.merge(df_mmoney_before_last, on="id"), on="id"
)

# Выводим информацию о датафрейме, чтобы проверить результат.
df_mmoney.info()

Теперь выполним аналогичные действия с датафреймом `df_mtime`.

In [ ]:
# Разбиваем df_mmoney на два датафрейма.
df_mtime_current = df_mtime[df_mtime.is_current_period == True]

df_mtime_last = df_mtime[df_mtime.is_current_period == False]

# Удаляем ненужный столбец и переименовываем нужный.
df_mtime_current = df_mtime_current.drop("is_current_period", axis=1).rename(
    columns={"minutes": "minutes_this_month"}
)

df_mtime_last = df_mtime_last.drop("is_current_period", axis=1).rename(
    columns={"minutes": "minutes_last_month"}
)

# Объединяем два датафрейма обратно.
df_mtime = df_mtime_current.merge(df_mtime_last, on="id")

# Выводим информацию о датафрейме, чтобы проверить результат.
df_mtime.info()

Теперь мы можем выполнить объединение трех датафреймов в один.

In [ ]:
# Объединяем три датафрейма в один.
df = df_mfile.merge(df_mmoney.merge(df_mtime, on="id"), on="id")

# Выводим информацию о датафрейме, чтобы проверить результат.
df.info()

**Вывод**: путем манипуляций с данными мы объединили три исходных датафрейма в один.

## Корреляционный анализ

В финальном датафрейме содержится 18 столбцов с данными. Для целей проведения корреляционного анализа мы можем исключить из рассмотрения стобец `id`.

Построим матрицу корреляций с использованием функционала библиотеки `phik`.

In [ ]:
# Создаем матрицу корреляции с тепловой картой.
plt.figure(figsize=(17, 5))
sns.heatmap(
    df.drop(["id"], axis=1).phik_matrix(
        interval_cols=(
            [
                "marketing_6_months",
                "marketing_this_month",
                "days_since_registration",
                "deal_share",
                "average_category_per_visit",
                "unpaid_goods_quarter",
                "errors",
                "pages_per_visit",
                "earnings_this_month",
                "earnings_last_month",
                "earnings_before_last",
                "minutes_this_month",
                "minutes_last_month",
            ]
        )
    ),
    annot=True,
    cmap="viridis",
)
plt.title("Матрица корреляции с тепловой картой датафрейма df")
plt.show()

Согласно полученной матрице и шкале Чеддока, мы наблюдаем в основном слабую (< 0,3) и умеренную (< 0,5) корреляцию между целевым признаком и предикторами, но в случае с `pages_per_visit` можно говорить о высокой (> 0,7) корреляции. В двух случаях (`allow_marketing` и `marketing_this_month`) корреляция вообще отсутствует.

Достаточно сильная зависимость наблюдается между `earnings_this_month` и `earnings_last_month`.

**Вывод**: корреляционный анализ показал достаточно слабые связи между целевым признаком и предикторами.

## Использование пайплайнов

На этапе преодбработки данных нами было произведено изменение типа данных для некоторых категориальных переменных на булев тип. С точки зрения кодирования категориальных признаков подобное изменение равносильно `OneHotEncoding`, поскольку дает нам один столбец, заполненный нулями (`False`) и единицами (`True`). В финальном датафрейме, на основании которого мы будем производить моделирование, остался только один категориальный столбец `popular_category`, который содержит наименование одной из шести категорий сайта. Мы также произведем кодирование данных в этом столбце с помощью `OneHotEncoder`.

Количественные столбцы мы пропустим через два метода масштабирования (`MinMaxScaler` и `StandardScaler`). Нам мог бы пригодиться `RobustScaler`, если бы в данных были выбросы, но единственный аномальный выброс мы удалили из рассмотрения.

Мы сравним четыре модели с выбранными случайно гиперпараметрами `DecisionTreeClassifier`, `KNeighborsClassifier`, `LogisticRegression` и `SVC`. Но прежде чем приступить к работе с моделями, необходимо остановиться на вопросе дисбаланса классов.

На этапе исследовательского анализа данных, нами было установлено, что целевой признак `same_level_activity` распределен неравномерно. После того, как мы удалили четыре записи из нашего датафрейма, мы имеем следующее соотношение: у 802 покупателей уровень активности на предыдущем уровне, у 494 покупателей он снизился. Модель, полученная на основании этих данных, будет с большей вероятностью предсказывать у покупателей сохранение прежнего уровня покупательской активости, а не ее снижение.

Есть несколько подходов к решению проблемы дисбаланса классов: уменьшение количества сэмлов мажорного класса, увеличение количества сэмплов минорного класса, синтез новых данных минорного класса. Поскольку мы работаем с не очень большим количеством наблюдений и разрыв в процентном соотношении велик, уменьшение количества сэмплов приведет к удалению большого количества данных. Увеличение количества сэмплов с использованием простого сэмплирования в нашем случае может привести к трех- и четырехкратному появлению одних и тех же данных в выборке.

Мы считаем, что в нашем случае оптимальным будет именно синтез новых данных. Перед нами стоит выбор между `SMOTE` и `ADASYN` из библиотеки `imbalanced-learn`. Использование `ADASYN` в нашей задаче было бы актуально, если бы нам уже была известно, что данные распределены равномерно и генерация с использованием `SMOTE` усложнит предсказания.

Таким образом, проблему дисбаланса мы будем решать при промощи `SMOTENC` в связи с наличием незакодированных категориальных данных.

In [ ]:
# Задаем константу.
RANDOM_STATE = 42

# Разбиваем данные на выборки.
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(["id", "same_level_activity"], axis=1),
    df["same_level_activity"],
    random_state=RANDOM_STATE,
    stratify=df["same_level_activity"],
)

# Категориальный столбец.
category = ["popular_category"]

# Задаем сэмплер.
sampler = SMOTENC(categorical_features=category, random_state=RANDOM_STATE)

# Проводим ресэмплинг путем синтеза данных.
X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)

Теперь, когда мы устранили дисбаланс классов в тренировочных данных, мы можем создать пайплайн, в котором будет осуществляться перебор моделей и выбор лучшей из них. Мы будем сравнивать модели по метрике $Accuracy$, поскольку данная метрика показывает долю верно классифицированных объектов ко всем классифированным объектам.

In [ ]:
# Создаем список с названиями количественных признаков.
num_columns = [
    "marketing_6_months",
    "days_since_registration",
    "deal_share",
    "average_category_per_visit",
    "unpaid_goods_quarter",
    "errors",
    "pages_per_visit",
    "earnings_this_month",
    "earnings_last_month",
    "earnings_before_last",
    "minutes_this_month",
    "minutes_last_month",
]

# Создаем пайплайн для подготовки признаков из списка ord_columns.
ohe_pipe = Pipeline([("ohe", OneHotEncoder(drop="first", sparse_output=False))])

# Создаем общий пайплайн для подготовки данных.
data_preprocessor = ColumnTransformer(
    [("ohe", ohe_pipe, category), ("num", MinMaxScaler(), num_columns)],
    remainder="passthrough",
)

# Создаем итоговый пайплайн: подготовка данных и модель.
pipe_final = Pipeline(
    [
        ("preprocessor", data_preprocessor),
        ("models", DecisionTreeClassifier(random_state=RANDOM_STATE)),
    ]
)

# Словари для моделей.
param_grid = [
    {
        "preprocessor__num": [StandardScaler(), MinMaxScaler()],
        "models": [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        "models__max_depth": range(2, 6),
        "models__max_features": range(2, 6),
    },
    {
        "preprocessor__num": [StandardScaler(), MinMaxScaler()],
        "models": [KNeighborsClassifier()],
        "models__n_neighbors": range(2, 9),
        "models__metric": ["euclidean", "cityblock"],
    },
    {
        "preprocessor__num": [StandardScaler(), MinMaxScaler()],
        "models": [LogisticRegression(random_state=RANDOM_STATE)],
        "models__C": [0.1, 1, 10],
        "models__solver": ["liblinear"],
        "models__penalty": ["l1", "l2"],
    },
    {
        "preprocessor__num": [StandardScaler(), MinMaxScaler()],
        "models": [SVC(random_state=RANDOM_STATE)],
        "models__kernel": ["linear", "rbf", "sigmoid", "poly"],
    },
]

# Поиск оптимальных параметров.
grid_search = GridSearchCV(pipe_final, param_grid, cv=5, scoring="accuracy", n_jobs=-1)

grid_search.fit(X_train_resampled, y_train_resampled)

# Выводим лучший результат и метрику.
print("Лучшая модель и ее параметры:\n\n", grid_search.best_estimator_, "\n\n")
print(
    f"Метрика Accuracy лучшей модели на тренировочной выборке: "
    f"{grid_search.best_score_:,.2f}".replace(".", ","),
    end=".\n",
)

# Проверям работу модели на тестовой выборке.
y_test_pred = grid_search.predict(X_test)
print(
    f"Метрика Accuracy лучшей модели на тестовой выборке: "
    f"{accuracy_score(y_test, y_test_pred):,.2f}".replace(".", ","),
    end=".",
)

**Вывод**: по итогам работы пайплайна с данными с устраненным дисбалансом классов мы получили результат, что лучшую метрику $Accuracy$ на тренировочных (0,88) и тестовых (0,84) данных показала модель kNN с высчитыванием манхэттенского расстояния для 7 соседей и стандартным масштабированием числовых признаков.

## Анализ важности признаков

Сохраним модель с полученными нами на прошлом шаге гиперпараметрами для детального анализа.

In [ ]:
# Задаем кодировщик для категориального столбца.
encoder = OneHotEncoder(drop="first", sparse_output=False)

# Проводим кодирование данных тренировочной и тестовой выборок.
# Тренировочная выборка прошла через сэмплирование SMOTENC.
X_train_model = (
    X_train_resampled.reset_index(drop=True)
    .join(
        pd.DataFrame(
            encoder.fit_transform(X_train_resampled[category]),
            columns=(
                "cosmetics",
                "kitchenware",
                "appliances",
                "beauty_and_health",
                "you_know_for_kids",
            ),
        ).astype(int)
    )
    .drop("popular_category", axis=1)
)

X_test_model = (
    X_test.reset_index(drop=True)
    .join(
        pd.DataFrame(
            encoder.fit_transform(X_test[category]),
            columns=(
                "cosmetics",
                "kitchenware",
                "appliances",
                "beauty_and_health",
                "you_know_for_kids",
            ),
        ).astype(int)
    )
    .drop("popular_category", axis=1)
)

# Задаем скейлер.
scaler = StandardScaler()

# Масштабируем числовые данные тренировочной и тестовой выборок.
X_train_model[num_columns] = scaler.fit_transform(X_train_model[num_columns])
X_test_model[num_columns] = scaler.transform(X_test_model[num_columns])

# Задаем модель с гиперпараметрами и обучаем ее.
model = KNeighborsClassifier(metric="cityblock", n_neighbors=7)
model.fit(X_train_model, y_train_resampled)

# Предсказываем целевой признак.
y_pred = model.predict(X_test_model)

# Убеждаемся, что получили ту же самую метрику.
print(
    f"Метрика Accuracy лучшей модели на тестовой выборке: "
    f"{accuracy_score(y_test, y_pred):,.2f}".replace(".", ","),
    end=".",
)

Посмотрим на матрицу ошибок нашей модели.

In [ ]:
# Строим матрицу ошибок.
c_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(17, 5))
sns.heatmap(c_matrix, annot=True, fmt="d", cmap="Blues_r")
plt.xlabel("Предсказанное значение")
plt.ylabel("Реальное значение")
plt.title("Покупательская активность \n")
plt.xticks([0.5, 1.5], ["Снизилась", "Прежняя"])
plt.yticks([0.5, 1.5], ["Снизилась", "Прежняя"])
plt.show()

Мы видим, что модель неплохо справляется с предсказанием классов на тестовых данных, однако нас интрересует более точное предсказание вероятности снижения покупательской активности в будущем, потому мы можем пожертвовать точностью в  предсказании True Positive значений. Построим кривые Precision и Recall и найдем оптимальное значение порога классификации для нашей модели.

In [ ]:
# Рассчитываем вероятности.
y_proba = model.predict_proba(X_test_model)[:, 1]

# Получаем значения.
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# Строим график.
plt.figure(figsize=(17, 5))
plt.plot(thresholds, precision[:-1])
plt.xlabel("Пороги")
plt.ylabel("Precision-Recall")
plt.plot(thresholds, recall[:-1])
plt.grid()
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()

# Выводим расчет оптимального значения.
print(
    f"Оптимальное значение порога для модели: "
    f"{round(thresholds[np.argmin(abs(precision - recall))], 5)}".replace(".", ","),
    end=".",
)

Подставим полученное нами оптимальное значение порога и построим новую матрицу ошибок.

In [ ]:
# Задаем новые значения.
y_proba_threshold = (y_proba > 0.57143).astype(int)

# Строим матрицу ошибок.
c_matrix_2 = confusion_matrix(y_test, y_proba_threshold)
plt.figure(figsize=(17, 5))
sns.heatmap(c_matrix_2, annot=True, fmt="d", cmap="Blues_r")
plt.xlabel("Предсказанное значение")
plt.ylabel("Реальное значение")
plt.title("Покупательская активность \n")
plt.xticks([0.5, 1.5], ["Снизилась", "Прежняя"])
plt.yticks([0.5, 1.5], ["Снизилась", "Прежняя"])
plt.show()

С такими настройками порога классификации модель способна точно предсказывать снижение активности в тестовой выборке в 105 случаях из 124 (\~ 84,7 %), упуская 19 случаев снижения покупательской активности. Таким образом, ошибка первого рода будет возникать в \~ 15,3 % случаев. С другой стороны ошибка второго рода будет возникать в 21 % случаев, и для бизнеса вероятнее всего выгоднее недопустить снижение активности клиентов, чем разослать лишние промокоды или специальные предложения тем, чья активность и так бы не снизилась.

В связи с тем, что оптимальной моделью по результатам работы пайплайна была выбрана модель kNN, мы не можем сделать интерпретацию признаков непосредственно из модели. При использовании библиотеки `shap` мы, к сожалению, не можем использовать `LinearExplainer` и должны обращаться к `KernelExplainer`, выполнение работы которого на нашем датасете займет несколько часов, потому мы можем взглянуть только на сэмпл данных, но и обработка 20 сэмплов все равно займет несколько минут (\~ 4 минуты на платформе Яндекса).

In [ ]:
# Инициализируем KernelExplainer из shap с сэмлпом из 100 наблюдений.
explainer = shap.KernelExplainer(model.predict, X_train_model.sample(100))

# Получаем значения для сэмпла из 20 наблюдений.
shap_values = explainer(X_train_model.sample(20))

Взглянем на графики для нашего сэмпла.

In [ ]:
# Строим графики.
shap.plots.beeswarm(shap_values)
shap.plots.bar(shap_values)

В соответствии с полученными графиками сложно говорить о высокой значимости признаков в нашем сэмпле. Наиболее важными признаками оказались `pages_per_visit`, `average_category_per_visit`, `minutes_last_month` и `minutes_this_month`. Далее идут `deal_share`, `popular_category` и `marketing_6_months`. *Графики сильно отличаются от сэмпла к сэмплу, описание может не совпадать с изображением выше.*

Если посмотреть на матрицу корреляции с тепловой картой, полученную нами на шаге 5 проекта, то коэффициент корреляции целевого признака и указанных признаков покажет примерно такую же картину.

**Вывод**: по результатам анализа мы можем сделать вывод, что наиболее важным показателем, влияющим на покупательскую активность и которым может управлять интернет-магазин, является объем маркетинговых коммуникаций за последние полгода. Также немалое значение имеет количество приообретенных клиентом товаров по акции. Можно предположить, что увеличение количества скидок клиентам, что положительно скажется на количестве приобретенных товаров, положительно скажется и на сохранении ими текущего уровня покупательской активности.

## Сегментация покупателей

Добавим в наш датафрейм данные из датафрейма `df_money`.

In [ ]:
# Объединяем датафреймы.
df_extra = df.merge(df_money, on="id")

# Выводим информацию о датафрейме, чтобы проверить результат.
df_extra.info()

Посмотрим на сегмент покупателей, которые покупают товары из категории «Товары для детей». В этой категории совершаются покупки не для себя, часто в подарок ребенку, так что мы считаем, что будет интересно посмотреть на то, как выглядят пользователи данной категории.

In [ ]:
# Выделяем данные в отдельный датафрейм.
df_segment = df_extra[df_extra["popular_category"] == "товары для детей"].drop(
    "popular_category", axis=1
)
df_segment.info()

Добавим в датафрейм дополнительный столбец, в котором будет отражено изменение выручки: если выручка от месяца к месяцу снижается, то поставим значение `decreased`, если увеличивается, то `increased`. В остальных случаях поставим значение `same`.

In [ ]:
# Сравниваем выручку по периодом.
for index, row in df_segment.iterrows():
    if (
        row["earnings_last_month"] < row["earnings_before_last"]
        and row["earnings_this_month"] < row["earnings_last_month"]
    ):
        df_segment.at[index, "earnings"] = "decreased"
    elif (
        row["earnings_last_month"] > row["earnings_before_last"]
        and row["earnings_this_month"] > row["earnings_last_month"]
    ):
        df_segment.at[index, "earnings"] = "increased"
    else:
        df_segment.at[index, "earnings"] = "same"

Теперь добавим информацию по количеству ошибок, с которыми пришлось столкнуться пользователям сервиса. Если их число больше среднего/медианного значения (\~ 4/4), то будем считать, что опыт использовния сайта для пользователя является скорее негативным (`negative`), чем позитивным (`positive`). 

In [ ]:
# Ранжируем пользовательский опыт взаимодействия с сайтом.
for index, row in df_segment.iterrows():
    if row["errors"] > 4:
        df_segment.at[index, "website_experience"] = "negative"
    else:
        df_segment.at[index, "website_experience"] = "positive"

Также посмотрим на то, как изменялись маркетинговые коммуникации с пользователями, стало ли их меньше (`decreased`), больше (`increased`) или осталось столько же (`same`), что и по сравнению с полугодом ранее.

In [ ]:
# Сравниваем маркетинговые коммуникации за два периода.
for index, row in df_segment.iterrows():
    if row["marketing_6_months"] > row["marketing_this_month"]:
        df_segment.at[index, "marketing"] = "decreased"
    elif row["marketing_6_months"] < row["marketing_this_month"]:
        df_segment.at[index, "marketing"] = "increased"
    else:
        df_segment.at[index, "marketing"] = "same"

Отберем пользователей, выручка от которых снизилась, маркетинговые коммуникации с которыми сократились, а их опыт пользования сайтом был негативным.

In [ ]:
# Отбираем пользователей из сегмента по критериям.
df_segment[
    (df_segment["earnings"] == "decreased")
    & (df_segment["marketing"] == "decreased")
    & (df_segment["website_experience"] == "negative")
]

Таких пользователей оказалось всего 2 из 329. Возможно, опыт пользования сайтом не имеет такого большого значения, как мы изначально предполагали. Посмотрим на срез без него.

In [ ]:
# Отбираем пользователей из сегмента по критериям.
df_segment[
    (df_segment["earnings"] == "decreased") & (df_segment["marketing"] == "decreased")
]

Мы получили срез из 14 пользователей, у 9 из которых покупательская активность уже снизилась, а у 5 она осталась на прежнем уровне. Посмотрим на них ближе.

In [ ]:
# Отбираем пользователей из сегмента по критериям.
display(
    df_segment[
        (df_segment["earnings"] == "decreased")
        & (df_segment["marketing"] == "decreased")
        & (df_segment["same_level_activity"] == True)
    ].describe()
)
df_segment[
    (df_segment["earnings"] == "decreased")
    & (df_segment["marketing"] == "decreased")
    & (df_segment["same_level_activity"] == True)
]

Мы видим, что у 2 пользователей в этой группе есть подписка на премиум, у 4 разрешены дополнительные маркетинговые коммуникации по интересующим их товарам, в среднем они зарегистрировались на сайте более двух лет назад. У пользоваталей без подписки больше покупок по акциям (от 15 до 40 %). Заинтересованность сайтом (количество минут) остается на прежнем уровне, или даже увеличивается в отдельных случаях, опыт пользования сайтом является «позитивным» (меньше ошибок, чем в среднем).

Несмотря на то, что пользователи в среднем просматривают 10-11 страниц за визит, количество их покупок снижается, а количество неоплаченных товаров в корзине минимально (среднее меньше 3, медиана — 3).

Мы считаем, что, возможно, имеет смысл ввести новый уровень подписки на сервис между бесплатной и премиальной, которая бы стимулировала пользователей совершать больше покупок и приносить большую выручку магазину.

Также имеет смысл рассмотреть некую программу лояльности, которая, например, позволяла бы пользователям копить баллы и тратить их на покупки в магазине, тем самым подталкивая их к покупкам в тех случаях, когда они сомневаются в необходимости совершить ее прямо сейчас. Мы имеем дело с устоявшейся аудиторией сайта и заинтересованы в том, чтобы это ядро продолжало приносить прибыль. Найти новых клиентов магазину сложнее, чем удержать старых.

Обязательно необходимо предусмотреть какие-то акции приуроченные к дню рождения ребенка, а также, возможно, предусмотреть дополнительную скидку в этот период на покупки для мамы, что могло бы положительно сказаться на среднем чеке. Для некоторых товаров категории «Товары для детей» можно было предложить скидку в зависимости от количества. При покупке определенного количества упаковок салфеток/подгузников/детского питания, следующая должна быть дешевле.

**Вывод**: мы считаем, что покупательская активность тех пользователей, что совершают покупки в основном в разделе «Товары для детей» сильно связана с тем, как часто на сайте проводятся какие-либо акции (розыгрыши призов, распродажи). Для удержания этой аудитории мы предлагаем рассмотреть вариант создания промежуточного уровня подписки между бесплатной и премиальной, введение программы лояльности для старых пользователей, дополнительные скидки, приуроченные к датам, или скидки, связанные с приобретенеием определенного количества одного и того же товара.

## Общий вывод

1. Мы произвели загрузку необходимых модулей и билиотек для работы, воспользовались собственной функцией для загрузки файлов с данными и произвели первичный (визуальный) анализ первых **20** строк в четырех таблицах. Нами были выявлены опечатки в некоторых столбцах. Исходя из количества записей в таблицах, мы предположили, что сервис предоставил информацию о **1 300** клиентах и их покупательской активности на сайте за три месяца. При этом в трех столбцах первой таблицы содержались данные за полгода, а во всех четырех таблицах столбец `id`, который позволил установить однозначное соотвествие между клиентом и информацией, собранной о нем в различных таблицах.
1. Мы произвели переименование столбцов в четырех датафреймах, сменили регистр в одном столбце датафрейма `df_mfile` и сменили тип данных в четырех столбцах двух датафреймов на булев. Мы выявили, что в данных не содержится полных дубликатов, что в каждом датафрейме есть данные ровно по **1 300** уникальным номерам покупателей в БД. Мы сделали допущение, что эти **1 300** номеров совпадают между датафреймами, поскольку данные в целом отличаются хорошим уровнем подготовки.
1. Мы рассмотрели данные во всех столбцах всех четырех датафреймов, произвели статистический анализ распределения данных, удалили **1** аномалию и **3** записи о клиентах, которые не совершали покупки на протяжении двух месяцев. Среднестатистический клиент интернет-магазина «В один клик» сохранил **прежний уровень покупательской активности**, **не имеет премиальной подписки** на сервис, **дал согласие** на получение дополнительной маркетинговой коммуникации по интересующим его товарам, приобрел товары в категории **«Товары для детей»**, **«Домашний текстиль»** или **«Косметика и аксессуары»**, в среднем имел **чуть более 4 коммуникаций** от маркетинговой службы интернет магазина (**4 коммуникации в текущем месяце**), зарегистрировался на сайте **около 600 дней назад**, **~ 24 % товаров** приобрел по акции, просматривал **3 категории за визит** на сайт в текущем месяце, имеет **~ 3 неоплаченных товара в корзине** за квартал, столкнулся с **~ 4 сбоями** во время визита сайта, просмотрел **~ 8 страниц товаров** за визит за квартал, принес магазину выручку **~ 4 тыс. руб.**, провел на сайте **~ 13 минут за визит**.
1. Путем манипуляций с данными мы объединили три исходных датафрейма в один.
1. Корреляционный анализ показал достаточно слабые связи между целевым признаком и предикторами.
1. По итогам работы пайплайна с данными с устраненным дисбалансом классов мы получили результат, что лучшую метрику $Accuracy$ на тренировочных (**0,88**) и тестовых (**0,84**) данных показала **модель kNN с высчитыванием манхэттенского расстояния для 7 соседей и стандартным масштабированием числовых признаков**.
1. По результатам анализа мы можем сделать вывод, что наиболее важным показателем, влияющим на покупательскую активность и которым может управлять интернет-магазин, является **объем маркетинговых коммуникаций за последние полгода**. Также немалое значение имеет **количество приообретенных клиентом товаров по акции**. Можно предположить, что **увеличение количества скидок** клиентам, что **положительно скажется на количестве приобретенных товаров**, **положительно скажется и на сохранении ими текущего уровня покупательской активности**.
1. По результатам рассмотрения сегмента покупателей пользователей, что совершают покупки в основном в разделе «Товары для детей», мы сделали вывод, что их **покупательская активность сильно связана с тем, как часто на сайте проводятся какие-либо акции (розыгрыши призов, распродажи)**. Для удержания этой аудитории мы предлагаем рассмотреть вариант **создания промежуточного уровня подписки между бесплатной и премиальной**, **введение программы лояльности для старых пользователей**, **дополнительные скидки**, приуроченные к датам, или скидки, связанные с приобретенеием определенного количества одного и того же товара.